<a href="https://colab.research.google.com/github/deskskeleton/diplomacy_chat_analysis/blob/main/discordchatanalysis_7doc2press.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

7DOC2 Chat Log Analysis Workbook

# Initial trial code

In [ ]:
from google.colab import files
uploaded = files.upload()  # select all the channel .json files


Saving 7 Days of Chaos 2 - Press - biblically-accurate-press [1450948009280934039] (2026-01-17 to 2026-01-18).json to 7 Days of Chaos 2 - Press - biblically-accurate-press [1450948009280934039] (2026-01-17 to 2026-01-18).json
Saving 7 Days of Chaos 2 - Press - hyperpeace-in-our-time [1450948510982733835] (2026-01-17 to 2026-01-18).json to 7 Days of Chaos 2 - Press - hyperpeace-in-our-time [1450948510982733835] (2026-01-17 to 2026-01-18).json
Saving 7 Days of Chaos 2 - Press - joining-the-hyperwar [1450933716460507266] (2026-01-17 to 2026-01-18).json to 7 Days of Chaos 2 - Press - joining-the-hyperwar [1450933716460507266] (2026-01-17 to 2026-01-18).json
Saving 7 Days of Chaos 2 - Press - riftposting [1450947315186532495] (2026-01-17 to 2026-01-18).json to 7 Days of Chaos 2 - Press - riftposting [1450947315186532495] (2026-01-17 to 2026-01-18).json
Saving 7 Days of Chaos 2 - Press - sea-of-ice [1450945904994685078] (2026-01-17 to 2026-01-18).json to 7 Days of Chaos 2 - Press - sea-of-ic

In [ ]:
import json
from pathlib import Path
import pandas as pd

def parse_dt(x):
    if x is None:
        return pd.NaT
    try:
        return pd.to_datetime(x)
    except Exception:
        return pd.NaT

def load_dce_json(path: Path):
    data = json.loads(path.read_text(encoding="utf-8"))
    msgs = data.get("messages") or data.get("Messages") or []

    channel = data.get("channel") or data.get("Channel") or {}
    channel_name = channel.get("name") or channel.get("Name") or path.stem

    rows = []
    for m in msgs:
        author = m.get("author") or m.get("Author") or {}
        ref = m.get("reference") or m.get("Reference") or {}

        rows.append({
            "channel": channel_name,
            "message_id": m.get("id") or m.get("Id"),
            "timestamp": parse_dt(m.get("timestamp") or m.get("Timestamp")),
            "author_name": author.get("name") or author.get("username") or author.get("Name") or author.get("Username"),
            "author_id": author.get("id") or author.get("Id"),
            "content": m.get("content") or m.get("Content") or "",
            "reply_to": (ref.get("messageId") or ref.get("MessageId")) if isinstance(ref, dict) else None,
        })

    df = pd.DataFrame(rows)
    df["source_file"] = path.name
    return df

paths = sorted(Path("/content").glob("*.json"))
dfs = []
for p in paths:
    try:
        dfs.append(load_dce_json(p))
    except Exception as e:
        print("Failed:", p.name, e)

all_msgs = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
all_msgs = all_msgs.dropna(subset=["timestamp"]).sort_values(["timestamp", "channel"]).reset_index(drop=True)

print("Files:", len(paths), "Messages:", len(all_msgs))
all_msgs.head(3)


Files: 7 Messages: 11265


,channel,message_id,timestamp,author_name,author_id,content,reply_to,source_file
0,sea-of-ice,1462174710346547394,2026-01-17 21:00:16.839000+01:00,radiolurken,216842183473364992,the sea of ice,None,7 Days of Chaos 2 - Press - sea-of-ice [145094...
1,joining-the-hyperwar,1462174711067836488,2026-01-17 21:00:17.011000+01:00,sleepykooka,961203159122968586,first,None,7 Days of Chaos 2 - Press - joining-the-hyperw...
2,sea-of-ice,1462174711092871329,2026-01-17 21:00:17.017000+01:00,spansos,655732888746196993,i like this channel call it sea of nice,None,7 Days of Chaos 2 - Press - sea-of-ice [145094...


In [ ]:
all_msgs["word_count"] = all_msgs["content"].fillna("").astype(str).str.split().str.len()

stats = (all_msgs.groupby("channel")
         .agg(
             messages=("message_id", "count"),
             unique_authors=("author_id", "nunique"),
             first_msg=("timestamp", "min"),
             last_msg=("timestamp", "max"),
             total_words=("word_count", "sum"),
         )
         .sort_values("messages", ascending=False))

stats


,messages,unique_authors,first_msg,last_msg,total_words
channel,,,,,
riftposting,3138,75,2026-01-17 21:00:19.059000+01:00,2026-01-17 23:59:59.045000+01:00,26222
joining-the-hyperwar,2273,71,2026-01-17 21:00:17.011000+01:00,2026-01-17 23:59:33.384000+01:00,15697
torus-supremacy,1835,64,2026-01-17 21:00:17.314000+01:00,2026-01-17 23:59:48.029000+01:00,12361
hyperpeace-in-our-time,1534,68,2026-01-17 21:00:17.906000+01:00,2026-01-17 23:59:59.418000+01:00,11977
stab-central,1362,65,2026-01-17 21:00:17.969000+01:00,2026-01-17 23:59:57.991000+01:00,8791
biblically-accurate-press,1082,60,2026-01-17 21:00:18.980000+01:00,2026-01-17 23:59:58.049000+01:00,8603
sea-of-ice,41,39,2026-01-17 21:00:16.839000+01:00,2026-01-17 23:50:10.023000+01:00,762


In [ ]:
top_overall = (all_msgs.groupby("author_name")
               .size()
               .sort_values(ascending=False)
               .head(20))
top_overall


,0
author_name,
vriska2195,674
jes_real,402
direapache,390
emperornaxos,343
aeolus_mare,336
jackofalltrades2751,297
styxsunset,283
of_many_voices,259
nevjev,246


In [ ]:
top_by_channel = (all_msgs.groupby(["channel","author_name"])
                  .size()
                  .reset_index(name="messages")
                  .sort_values(["channel","messages"], ascending=[True, False]))

top_by_channel.groupby("channel").head(10)


,channel,author_name,messages
16,biblically-accurate-press,direapache,164
19,biblically-accurate-press,electrobis,83
45,biblically-accurate-press,sanserino34,70
10,biblically-accurate-press,ch4tt3rb0x,63
21,biblically-accurate-press,epicsquidmoment,61
...,...,...,...
408,torus-supremacy,josogamer1029,100
384,torus-supremacy,celestial_omega1,78
382,torus-supremacy,aeolus_mare,77
402,torus-supremacy,jacket08,63


In [ ]:
replies = all_msgs.dropna(subset=["reply_to"])
reply_counts = replies.groupby(["channel","reply_to"]).size().reset_index(name="reply_count")

key_msgs = (all_msgs.merge(reply_counts, how="left",
                           left_on=["channel","message_id"],
                           right_on=["channel","reply_to"])
            .drop(columns=["reply_to_y"])
            .rename(columns={"reply_to_x":"reply_to"})
            .fillna({"reply_count": 0})
            .sort_values(["channel","reply_count","timestamp"], ascending=[True, False, True]))

# Show top 10 most replied-to messages per channel
key_msgs.groupby("channel").head(10)[["channel","timestamp","author_name","reply_count","content"]]


,channel,timestamp,author_name,reply_count,content
13,biblically-accurate-press,2026-01-17 21:00:20.260000+01:00,electrobis,4.0,WHEELERS WE MUST MEET AT THIS ROUND TABLE\n@Ze...
3795,biblically-accurate-press,2026-01-17 21:46:12.968000+01:00,electrobis,4.0,The Wheel Alliance Outline:\n1. The Wheel Alli...
4482,biblically-accurate-press,2026-01-17 21:55:56.279000+01:00,rubercuber,4.0,the gang
331,biblically-accurate-press,2026-01-17 21:03:44.899000+01:00,ch4tt3rb0x,3.0,@Constantinople @Şanlıurfa fellow members of t...
1733,biblically-accurate-press,2026-01-17 21:20:00.382000+01:00,coolschnoodle,3.0,I'd be happy to do that if I get a dot and the...
...,...,...,...,...,...
5742,torus-supremacy,2026-01-17 22:14:15.999000+01:00,jacket08,4.0,Holy shit super canada
6056,torus-supremacy,2026-01-17 22:18:25.608000+01:00,jes_real,4.0,@Auckland @Dover @Līhuʻe @Manchester @Queensto...
6196,torus-supremacy,2026-01-17 22:20:12.244000+01:00,coolschnoodle,4.0,@The Grouse | STFU @L’emperur Bursa[STFU] @Ru...
6224,torus-supremacy,2026-01-17 22:20:41.438000+01:00,jes_real,4.0,I also think we shouldn't have a leader


In [ ]:
import re
from collections import Counter

def top_terms(texts, k=20):
    tokens = []
    for t in texts:
        t = (t or "").lower()
        t = re.sub(r"http\S+", "", t)
        t = re.sub(r"[^a-z0-9\s#]+", " ", t)
        toks = [x for x in t.split() if len(x) >= 3 and not x.isdigit()]
        tokens.extend(toks)
    return Counter(tokens).most_common(k)

for ch in stats.index.tolist():
    terms = top_terms(all_msgs.loc[all_msgs["channel"]==ch, "content"].tolist(), k=25)
    print("\n==", ch, "==")
    print(", ".join([f"{w}({c})" for w,c in terms[:20]]))



== riftposting ==
the(841), you(410), and(382), that(268), austria(254), for(245), are(227), alliance(207), with(194), have(191), but(186), not(171), think(164), like(143), yeah(141), this(140), just(139), can(139), pal(131), what(122)

== joining-the-hyperwar ==
the(476), you(274), and(221), that(182), for(157), have(134), not(124), this(119), are(118), nantes(115), with(107), alliance(104), just(94), can(93), but(89), they(88), france(87), what(87), get(81), think(75)

== torus-supremacy ==
the(350), you(232), are(139), and(135), that(124), for(120), with(109), but(100), they(90), alliance(89), have(87), what(75), think(73), this(69), not(69), can(67), yeah(62), all(60), just(57), good(51)

== hyperpeace-in-our-time ==
the(346), you(171), and(158), that(152), alliance(116), for(105), have(96), with(96), are(94), turkey(93), but(89), this(82), what(79), they(75), not(75), think(74), just(72), yeah(68), all(67), like(65)

== stab-central ==
the(313), and(145), you(139), are(91), that(

In [ ]:
def clip(s, n=280):
    s = (s or "").strip().replace("\n", " ")
    return s if len(s) <= n else s[:n-1] + "…"

digest_lines = []
for ch, row in stats.iterrows():
    digest_lines.append(f"## #{ch}")
    digest_lines.append(f"- Messages: **{int(row['messages'])}** | Speakers: **{int(row['unique_authors'])}**")
    digest_lines.append(f"- Window: **{row['first_msg']} → {row['last_msg']}**")
    digest_lines.append("")
    digest_lines.append("**Most replied-to messages (top 5):**")

    km = key_msgs[key_msgs["channel"]==ch].head(5)
    if km.empty:
        digest_lines.append("- (none)")
    else:
        for _, r in km.iterrows():
            digest_lines.append(f"- ({int(r['reply_count'])} replies) **{r['author_name']}**: {clip(r['content'])}")

    digest_lines.append("")
    # top speakers
    speakers = (all_msgs[all_msgs["channel"]==ch]
                .groupby("author_name").size()
                .sort_values(ascending=False).head(5))
    digest_lines.append("**Top speakers (top 5):** " + ", ".join([f"{name} ({cnt})" for name, cnt in speakers.items()]))
    digest_lines.append("\n---\n")

newsletter = "\n".join(digest_lines)
print(newsletter[:2000])  # preview


## #riftposting
- Messages: **3138** | Speakers: **75**
- Window: **2026-01-17 21:00:19.059000+01:00 → 2026-01-17 23:59:59.045000+01:00**

**Most replied-to messages (top 5):**
- (6 replies) **andrium**: please CAN ANYBODY TELL ME WHAT IS GOING ON IN THEIR RESPECTIVE AUSTRIAS
- (5 replies) **keni130600**: I dont think there are many people who can understand what happening here
- (5 replies) **samosth**: @Traun @Leonding @Wiener Neustadt @Sankt Pölten @Klagenfurt just wanted to say hi to you guys too! I know we'll be the real alliance here as the hungary gang
- (5 replies) **swiliu**: hows the austrias looking?
- (5 replies) **nevjev**: 

**Top speakers (top 5):** huxrax (230), nevjev (219), aeolus_mare (212), ejvind2946 (204), _bumble (193)

---

## #joining-the-hyperwar
- Messages: **2273** | Speakers: **71**
- Window: **2026-01-17 21:00:17.011000+01:00 → 2026-01-17 23:59:33.384000+01:00**

**Most replied-to messages (top 5):**
- (7 replies) **armygeneralf**: updated map
- (5 replies

In [ ]:
from pathlib import Path
Path("press_digest_opening_night.md").write_text(newsletter, encoding="utf-8")
files.download("press_digest_opening_night.md")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# LLM analysis

In [ ]:
!pip -q install openai pydantic pandas python-dateutil


set an openai api key in secrets or .env

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")



upload files

In [ ]:
from google.colab import files
uploaded = files.upload()  # select all channel JSON files


Saving 7 Days of Chaos 2 - Press - biblically-accurate-press [1450948009280934039] (2026-01-17 to 2026-01-18).json to 7 Days of Chaos 2 - Press - biblically-accurate-press [1450948009280934039] (2026-01-17 to 2026-01-18) (1).json
Saving 7 Days of Chaos 2 - Press - hyperpeace-in-our-time [1450948510982733835] (2026-01-17 to 2026-01-18).json to 7 Days of Chaos 2 - Press - hyperpeace-in-our-time [1450948510982733835] (2026-01-17 to 2026-01-18) (1).json
Saving 7 Days of Chaos 2 - Press - joining-the-hyperwar [1450933716460507266] (2026-01-17 to 2026-01-18).json to 7 Days of Chaos 2 - Press - joining-the-hyperwar [1450933716460507266] (2026-01-17 to 2026-01-18) (1).json
Saving 7 Days of Chaos 2 - Press - riftposting [1450947315186532495] (2026-01-17 to 2026-01-18).json to 7 Days of Chaos 2 - Press - riftposting [1450947315186532495] (2026-01-17 to 2026-01-18) (1).json
Saving 7 Days of Chaos 2 - Press - sea-of-ice [1450945904994685078] (2026-01-17 to 2026-01-18).json to 7 Days of Chaos 2 - P

load and normalize messages

In [ ]:
import json
from pathlib import Path
import pandas as pd

def parse_dt(x):
    try:
        return pd.to_datetime(x)
    except Exception:
        return pd.NaT

def load_dce_json(path: Path) -> pd.DataFrame:
    data = json.loads(path.read_text(encoding="utf-8"))
    msgs = data.get("messages") or data.get("Messages") or []

    channel = data.get("channel") or data.get("Channel") or {}
    channel_name = channel.get("name") or channel.get("Name") or path.stem

    rows = []
    for m in msgs:
        author = m.get("author") or m.get("Author") or {}
        ref = m.get("reference") or m.get("Reference") or {}

        rows.append({
            "channel": channel_name,
            "message_id": m.get("id") or m.get("Id"),
            "timestamp": parse_dt(m.get("timestamp") or m.get("Timestamp")),
            "author_name": author.get("name") or author.get("username") or author.get("Name") or author.get("Username"),
            "author_id": author.get("id") or author.get("Id"),
            "content": (m.get("content") or m.get("Content") or ""),
            "reply_to": (ref.get("messageId") or ref.get("MessageId")) if isinstance(ref, dict) else None,
        })

    df = pd.DataFrame(rows)
    df["source_file"] = path.name
    return df

paths = sorted(Path("/content").glob("*.json"))
dfs = [load_dce_json(p) for p in paths]
all_msgs = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

all_msgs = all_msgs.dropna(subset=["timestamp"]).sort_values(["channel","timestamp"]).reset_index(drop=True)
print("Files:", len(paths), "Messages:", len(all_msgs))
all_msgs.head(3)


Files: 14 Messages: 22530


,channel,message_id,timestamp,author_name,author_id,content,reply_to,source_file
0,biblically-accurate-press,1462174719326552218,2026-01-17 21:00:18.980000+01:00,thorny_riot0079,1374702267541753957,I claim this channel in the name of Türkiye an...,None,7 Days of Chaos 2 - Press - biblically-accurat...
1,biblically-accurate-press,1462174719326552218,2026-01-17 21:00:18.980000+01:00,thorny_riot0079,1374702267541753957,I claim this channel in the name of Türkiye an...,None,7 Days of Chaos 2 - Press - biblically-accurat...
2,biblically-accurate-press,1462174724695130112,2026-01-17 21:00:20.260000+01:00,electrobis,788873432980652093,WHEELERS WE MUST MEET AT THIS ROUND TABLE\n@Ze...,None,7 Days of Chaos 2 - Press - biblically-accurat...


merge bursty consecutive messages (per channel + author)

In [ ]:
from datetime import timedelta

MERGE_WITHIN_SECONDS = 180  # 3 minutes

def merge_bursts(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(["channel", "timestamp"]).copy()
    out_rows = []

    current = None
    for row in df.itertuples(index=False):
        if current is None:
            current = row._asdict()
            current["content"] = str(current["content"] or "")
            continue

        same_channel = (row.channel == current["channel"])
        same_author = (row.author_id == current["author_id"])
        close_in_time = (row.timestamp - current["timestamp"] <= pd.Timedelta(seconds=MERGE_WITHIN_SECONDS))

        if same_channel and same_author and close_in_time:
            # extend block
            current["content"] += "\n" + str(row.content or "")
            current["timestamp_end"] = row.timestamp
        else:
            # finalize previous
            current.setdefault("timestamp_end", current["timestamp"])
            out_rows.append(current)
            current = row._asdict()
            current["content"] = str(current["content"] or "")

    if current is not None:
        current.setdefault("timestamp_end", current["timestamp"])
        out_rows.append(current)

    merged = pd.DataFrame(out_rows)
    merged["timestamp_end"] = pd.to_datetime(merged["timestamp_end"])
    return merged.reset_index(drop=True)

merged = merge_bursts(all_msgs)

print("Merged turns:", len(merged))
merged.head(3)


Merged turns: 10144


,channel,message_id,timestamp,author_name,author_id,content,reply_to,source_file,timestamp_end
0,biblically-accurate-press,1462174719326552218,2026-01-17 21:00:18.980000+01:00,thorny_riot0079,1374702267541753957,I claim this channel in the name of Türkiye an...,None,7 Days of Chaos 2 - Press - biblically-accurat...,2026-01-17 21:00:18.980000+01:00
1,biblically-accurate-press,1462174724695130112,2026-01-17 21:00:20.260000+01:00,electrobis,788873432980652093,WHEELERS WE MUST MEET AT THIS ROUND TABLE\n@Ze...,None,7 Days of Chaos 2 - Press - biblically-accurat...,2026-01-17 21:00:20.260000+01:00
2,biblically-accurate-press,1462174807717318667,2026-01-17 21:00:40.054000+01:00,credencetothekind,1245693791730536530,:Summer_Ray_Yes:\n:Summer_Ray_Yes:\nFULL SUPPO...,1462174724695130112,7 Days of Chaos 2 - Press - biblically-accurat...,2026-01-17 21:00:46.692000+01:00


chunk per channel

In [ ]:
from typing import List, Dict

TURNS_PER_CHUNK = 120  # adjust if needed

def make_chunks(merged_df: pd.DataFrame) -> List[Dict]:
    chunks = []
    for ch, sub in merged_df.groupby("channel"):
        sub = sub.sort_values("timestamp")
        turns = sub.to_dict(orient="records")
        for i in range(0, len(turns), TURNS_PER_CHUNK):
            part = turns[i:i+TURNS_PER_CHUNK]
            chunks.append({
                "channel": ch,
                "start": part[0]["timestamp"],
                "end": part[-1].get("timestamp_end", part[-1]["timestamp"]),
                "turns": part,
            })
    return chunks

chunks = make_chunks(merged)
print("Chunks:", len(chunks))
chunks[0]["channel"], chunks[0]["start"], chunks[0]["end"], len(chunks[0]["turns"])


Chunks: 88


('biblically-accurate-press',
 Timestamp('2026-01-17 21:00:18.980000+0100', tz='UTC+01:00'),
 Timestamp('2026-01-17 21:19:42.600000+0100', tz='UTC+01:00'),
 120)

stage 1 extraction schema

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Literal, Optional

EventType = Literal["proposal","commitment","request","info","warning","meta","other"]

class ExtractedEvent(BaseModel):
    type: EventType
    summary: str = Field(description="Short, neutral summary of the event (1–2 sentences).")
    actors: List[str] = Field(default_factory=list, description="Named players/powers if explicitly mentioned; otherwise empty.")
    evidence: str = Field(description="A short snippet or near-quote (<=200 chars) grounding the event.")
    confidence: float = Field(ge=0.0, le=1.0, description="Model confidence from 0 to 1.")
    timestamp_hint: Optional[str] = Field(default=None, description="Optional ISO timestamp if obvious from context.")

class ExtractionResult(BaseModel):
    channel: str
    window_start: str
    window_end: str
    events: List[ExtractedEvent]


stage 1 extraction calls (async, one per chunk)

In [ ]:
import asyncio
from openai import AsyncOpenAI

MODEL_EXTRACT = "gpt-5.2"  # swap if desired

client = AsyncOpenAI()

def format_chunk_for_model(chunk) -> str:
    lines = []
    for t in chunk["turns"]:
        ts = str(t["timestamp"])
        author = t.get("author_name") or "unknown"
        content = (t.get("content") or "").strip()
        if not content:
            continue
        lines.append(f"[{ts}] {author}: {content}")
    return "\n".join(lines)

EXTRACTION_SYSTEM = """You extract structured events from a Discord press channel log.
Be neutral. Do not infer hidden motives. Only use what is explicitly stated.
Prefer fewer, higher-signal events over many tiny ones.
Evidence must be a short snippet from the text (<=200 chars)."""

EXTRACTION_USER_TEMPLATE = """Channel: {channel}
Window: {start} to {end}

Log:
{log}

Task:
Extract the key events as a compact list: proposals, commitments, requests, notable info/claims, warnings, or meta logistics.
Return only events that a reader would want in a daily digest.
"""

sem = asyncio.Semaphore(3)  # concurrency limit; raise slowly if you want

async def extract_one(chunk):
    async with sem:
        log_text = format_chunk_for_model(chunk)
        prompt = EXTRACTION_USER_TEMPLATE.format(
            channel=chunk["channel"],
            start=str(chunk["start"]),
            end=str(chunk["end"]),
            log=log_text
        )

        resp = await client.responses.parse(
            model=MODEL_EXTRACT,
            input=[
                {"role": "system", "content": EXTRACTION_SYSTEM},
                {"role": "user", "content": prompt},
            ],
            # Optional reasoning control (if supported): keep it modest
            # reasoning={"effort":"low"},
            text_format=ExtractionResult,
        )
        out = resp.output_parsed
        return out.model_dump()

async def run_extraction(chunks):
    tasks = [extract_one(c) for c in chunks]
    results = []
    for coro in asyncio.as_completed(tasks):
        results.append(await coro)
    return results

extraction_results = await run_extraction(chunks)
len(extraction_results), extraction_results[0].keys()


(88, dict_keys(['channel', 'window_start', 'window_end', 'events']))

In [ ]:
import statistics

# after extraction_results exists
events_per_chunk = [len(r["events"]) for r in extraction_results]
print("Chunks:", len(events_per_chunk))
print("Events total:", sum(events_per_chunk))
print("Mean events/chunk:", statistics.mean(events_per_chunk))
print("Median events/chunk:", statistics.median(events_per_chunk))
print("P90 events/chunk:", sorted(events_per_chunk)[int(0.9*len(events_per_chunk))-1])


Chunks: 88
Events total: 1134
Mean events/chunk: 12.886363636363637
Median events/chunk: 13.0
P90 events/chunk: 16


In [ ]:
import json
from pathlib import Path

Path("extraction_results.json").write_text(json.dumps(extraction_results, indent=2), encoding="utf-8")


521878

consolidate events (dedupe lightly)

In [ ]:
import pandas as pd

events = []
for r in extraction_results:
    for e in r["events"]:
        events.append({
            "channel": r["channel"],
            "window_start": r["window_start"],
            "window_end": r["window_end"],
            **e
        })

events_df = pd.DataFrame(events)
print("Events extracted:", len(events_df))
events_df.head(10)


Events extracted: 1134


,channel,window_start,window_end,type,summary,actors,evidence,confidence,timestamp_hint
0,riftposting,2026-01-17 21:04:53.396000+01:00,2026-01-17 21:08:54.764000+01:00,proposal,A player proposed forming an alliance/working ...,[aeolus_mare],"""Any interest in allying together here? Doesn’...",0.78,2026-01-17T21:04:56.435000+01:00
1,riftposting,2026-01-17 21:04:53.396000+01:00,2026-01-17 21:08:54.764000+01:00,request,A headcount was requested for how many players...,[celestial_omega1],"""Can we get a headcount of how many people on ...",0.86,2026-01-17T21:05:29.676000+01:00
2,riftposting,2026-01-17 21:04:53.396000+01:00,2026-01-17 21:08:54.764000+01:00,proposal,Several users discussed forming an Austria-foc...,"[keni130600, tigera, skurkmedw, bombadii, aeol...","""We should have allience like PAL"" / ""PAL core...",0.84,2026-01-17T21:06:48.962000+01:00
3,riftposting,2026-01-17 21:04:53.396000+01:00,2026-01-17 21:08:54.764000+01:00,warning,Concerns were raised that a full Austria mega-...,"[aeolus_mare, kwahwodahs, undercookedfish]","""I don’t want a full Austria alliance"" / ""unit...",0.80,2026-01-17T21:08:31.472000+01:00
4,riftposting,2026-01-17 21:04:53.396000+01:00,2026-01-17 21:08:54.764000+01:00,proposal,A rule proposal: if Austria forms a big allian...,[hemag],"""people who NMR during the first 2 or 3 years ...",0.87,2026-01-17T21:07:23.511000+01:00
5,riftposting,2026-01-17 21:04:53.396000+01:00,2026-01-17 21:08:54.764000+01:00,request,A group was asked whether they should join PAL...,[ejvind2946],"""Should we join PAL (Pan-Austrian league) or c...",0.84,2026-01-17T21:07:10.978000+01:00
6,riftposting,2026-01-17 21:04:53.396000+01:00,2026-01-17 21:08:54.764000+01:00,request,West German players were asked about being fri...,[crazycoolin],"""hey west germans how do you feel about being ...",0.84,2026-01-17T21:07:21.498000+01:00
7,riftposting,2026-01-17 21:04:53.396000+01:00,2026-01-17 21:08:54.764000+01:00,commitment,A player stated they would like to work with e...,[nevjev],"""I WOULD LIKE TO WORK WITH EVERYONE IN AUSTRIA""",0.77,2026-01-17T21:08:42.970000+01:00
8,riftposting,2026-01-17 21:04:53.396000+01:00,2026-01-17 21:08:54.764000+01:00,commitment,A player said they were happy to be allied wit...,[strangedecoder],"""I'm happy to be allied to Austria 4, even tho...",0.77,2026-01-17T21:08:33.273000+01:00
9,riftposting,2026-01-17 21:04:53.396000+01:00,2026-01-17 21:08:54.764000+01:00,warning,A user warned they would retaliate if anyone “...,"[emperornaxos, Varna]","""if anyone messes with my homie Varna im nukin...",0.75,2026-01-17T21:08:49.179000+01:00


optional quick dedupe (same channel/type/summary)

In [ ]:
events_df["summary_norm"] = events_df["summary"].str.lower().str.strip()
events_df = events_df.sort_values(["channel","type","confidence"], ascending=[True, True, False])
events_df = events_df.drop_duplicates(subset=["channel","type","summary_norm"], keep="first").reset_index(drop=True)
print("After dedupe:", len(events_df))


Stage 2 synthesis (markdown digest)

In [ ]:
TYPE_CAPS = {
    "proposal": 12,
    "commitment": 10,
    "request": 10,
    "info": 8,
    "warning": 6,
    "meta": 6,
    "other": 6,
}

def cap_events(df):
    keep = []
    for (ch, t), sub in df.groupby(["channel","type"]):
        cap = TYPE_CAPS.get(t, 6)
        keep.append(sub.sort_values("confidence", ascending=False).head(cap))
    return pd.concat(keep, ignore_index=True) if keep else df

events_capped = cap_events(events_df)
print("Capped events:", len(events_capped))
events_capped.groupby(["channel","type"]).size().sort_values(ascending=False).head(20)


Capped events: 317


channel                    type      
biblically-accurate-press  proposal      12
riftposting                proposal      12
joining-the-hyperwar       proposal      12
torus-supremacy            proposal      12
stab-central               proposal      12
hyperpeace-in-our-time     proposal      12
                           commitment    10
biblically-accurate-press  request       10
stab-central               commitment    10
                           request       10
torus-supremacy            request       10
                           commitment    10
joining-the-hyperwar       request       10
riftposting                request       10
                           commitment    10
joining-the-hyperwar       commitment    10
hyperpeace-in-our-time     request       10
biblically-accurate-press  commitment     9
                           info           8
hyperpeace-in-our-time     info           8
dtype: int64

In [ ]:
from openai import OpenAI
client_sync = OpenAI()

MODEL_SYNTH = "gpt-5.2"

channels = sorted(events_capped["channel"].unique().tolist())

def bullets_for_channel(df, channel):
    sub = df[df["channel"]==channel].sort_values(["type","confidence"], ascending=[True, False])
    lines = []
    for r in sub.itertuples(index=False):
        actors = ", ".join(r.actors) if isinstance(r.actors, list) and r.actors else ""
        ev = f"- [{r.type}] {r.summary}"
        if actors:
            ev += f" (actors: {actors})"
        ev += f" (conf {r.confidence:.2f})"
        lines.append(ev)
    return "\n".join(lines)

per_channel = []
for ch in channels:
    per_channel.append(f"### #{ch}\n{bullets_for_channel(events_capped, ch)}\n")
synthesis_input = "\n".join(per_channel)

SYNTHESIS_SYSTEM = """Write a daily press digest for a Diplomacy-like game from extracted events.

Rules:
- Use ONLY the provided events; do not invent details.
- Be neutral and concrete.
- For EACH channel, include these sections in this order:
  1) Key proposals (max 8 bullets)
  2) Confirmed commitments (max 6 bullets)
  3) Open questions / pending decisions (max 6 bullets)
  4) Notable claims / intel (max 5 bullets)
  5) Meta / logistics (max 3 bullets)

- End with a short Cross-channel section:
  - "What’s converging" (max 5 bullets)
  - "What’s conflicting/unclear" (max 5 bullets)
  - "What to watch next" (max 5 bullets)

Write in Markdown, ready to paste to Discord.
"""

resp = client_sync.responses.create(
    model=MODEL_SYNTH,
    input=[
        {"role":"system","content": SYNTHESIS_SYSTEM},
        {"role":"user","content": synthesis_input},
    ],
)

digest_md = resp.output_text
print(digest_md[:2000])


## #biblically-accurate-press

### 1) Key proposals
- Form an “Alliance of the Wheel” / Wheel peace pact to prevent infighting and expand outward. (electrobis, Sacland)
- Adopt Wheel “constitution” principles: democratic decision-making, no fratricide, mutual defense/collective retaliation for stabs; fleets-only posture with armies only by vote. (electrobis, ronnec_15723, sanserino34, nopesauc3)
- Encourage Mystery to join the Wheel alliance. (andrium, ronnec_15723)
- Direapache asks whether L’March should run an independent eastern plan or coordinate with other French members; styxsunset supports decentralized planning. (direapache, Nimes, AdapazarıMouseMan, styxsunset)
- Propose a defensive pact with southern Turkish players to deter/answer attacks by other Turkish players; strong support signaled. (direapache, swiliu)
- Draft “First Decree of the Republic”: claims Nice is rallying to attack Nantes; proposes “death of Nice” and that Reims must waive. (styxsunset)
- Discuss an anti-Ni

save/download digest + extracted events

In [ ]:
len(digest_md), digest_md.count("\n"), digest_md[-500:]


(24158,
 300,
 'y alliance solidifies and how it interacts with STFU and Flightless Birds, especially around builds (Mersin fleet vs army). (#hyperpeace-in-our-time)\n- Whether Austria coordination resolves into PAL/Cross-Austria vs smaller regional groupings like the Bastion, amid anti-mega sentiment. (#riftposting; #sea-of-ice)\n- Any escalation signals around France (broadcast “invading France” and other external commitments) and how they intersect with ongoing alliance formation. (#stab-central; #riftposting)')

In [ ]:
from pathlib import Path
from google.colab import files

Path("press_digest_opening_night.md").write_text(digest_md, encoding="utf-8")
files.download("press_digest_opening_night.md")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from pathlib import Path
from google.colab import files

Path("events_extracted.csv").write_text(events_df.to_csv(index=False), encoding="utf-8")
Path("press_digest.md").write_text(digest_md, encoding="utf-8")

files.download("press_digest.md")
files.download("events_extracted.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from pathlib import Path
import re

# ---- config ----
INPUT_MD = "press_digest.md"          # change if needed
OUT_DIR = Path("discord_parts")
MAX_CHARS = 1900                      # Discord is ~2000; keep margin
HEADER_PREFIX = ""                    # e.g., "**Daily Press Digest**\n" if you want a prefix on every part
ADD_PART_FOOTER = True                # adds "Part X/Y" footer to each chunk

# ---- helpers ----
def read_text(path):
    return Path(path).read_text(encoding="utf-8")

def split_by_heading(md: str):
    """
    Split at '## ' headings. Keeps the heading with its section.
    If there is preamble text before first ##, it becomes its own block.
    """
    lines = md.splitlines(True)  # keep newlines
    blocks = []
    cur = []
    for ln in lines:
        if ln.startswith("## "):
            if cur:
                blocks.append("".join(cur).strip())
                cur = []
        cur.append(ln)
    if cur:
        blocks.append("".join(cur).strip())
    return [b for b in blocks if b]

def soft_split_long_block(block: str, max_chars: int):
    """
    If a single block is too long, split it by:
    1) '### ' subheadings
    2) blank-line paragraphs
    3) hard line breaks
    """
    if len(block) <= max_chars:
        return [block]

    # 1) split by ### subheadings
    parts = re.split(r'(?m)^(?=### )', block)
    parts = [p.strip() for p in parts if p.strip()]
    if len(parts) > 1:
        return pack_parts(parts, max_chars)

    # 2) split by paragraphs
    paras = re.split(r'\n\s*\n', block.strip())
    if len(paras) > 1:
        paras = [p.strip() for p in paras if p.strip()]
        return pack_parts(paras, max_chars, joiner="\n\n")

    # 3) split by lines (last resort)
    lines = block.splitlines()
    return pack_parts(lines, max_chars, joiner="\n")

def pack_parts(parts, max_chars: int, joiner="\n\n"):
    """
    Greedy packer: combines parts into chunks up to max_chars.
    """
    chunks = []
    cur = ""
    for p in parts:
        candidate = p if not cur else (cur + joiner + p)
        if len(candidate) <= max_chars:
            cur = candidate
        else:
            if cur:
                chunks.append(cur.strip())
                cur = p
            else:
                # Single piece still too big; hard-split it
                chunks.extend(hard_split_text(p, max_chars))
                cur = ""
    if cur:
        chunks.append(cur.strip())
    return chunks

def hard_split_text(text: str, max_chars: int):
    """
    Hard split a string into max_chars slices (rarely needed).
    """
    text = text.strip()
    return [text[i:i+max_chars].strip() for i in range(0, len(text), max_chars)]

def build_chunks(md: str, max_chars: int):
    # Split by major headings
    blocks = split_by_heading(md)

    # Ensure no block exceeds max_chars by soft splitting
    normalized = []
    for b in blocks:
        normalized.extend(soft_split_long_block(b, max_chars))

    # Pack into final chunks (so multiple sections can fit in one message if small)
    chunks = []
    cur = ""
    for b in normalized:
        b = b.strip()
        if not b:
            continue
        candidate = b if not cur else (cur + "\n\n" + b)
        if len(candidate) <= max_chars:
            cur = candidate
        else:
            if cur:
                chunks.append(cur.strip())
            # If b is still too big (shouldn't be), hard split
            if len(b) > max_chars:
                chunks.extend(hard_split_text(b, max_chars))
                cur = ""
            else:
                cur = b
    if cur:
        chunks.append(cur.strip())
    return chunks

# ---- run ----
md = read_text(INPUT_MD)
raw_chunks = build_chunks(md, MAX_CHARS - (len(HEADER_PREFIX) + 40 if ADD_PART_FOOTER else len(HEADER_PREFIX)))

# Add header prefix + part footer
total = len(raw_chunks)
chunks = []
for i, c in enumerate(raw_chunks, start=1):
    out = c
    if HEADER_PREFIX:
        out = HEADER_PREFIX.strip() + "\n\n" + out
    if ADD_PART_FOOTER:
        out = out.rstrip() + f"\n\n_(Part {i}/{total})_"
    chunks.append(out)

OUT_DIR.mkdir(parents=True, exist_ok=True)
for i, c in enumerate(chunks, start=1):
    (OUT_DIR / f"discord_part_{i:02d}.txt").write_text(c, encoding="utf-8")

print(f"Wrote {len(chunks)} parts to: {OUT_DIR.resolve()}")
for i, c in enumerate(chunks, start=1):
    preview = c[:300].replace("\n", "\\n")
    print(f"{i:02d}/{total}: {len(c)} chars | {preview}...")


Wrote 17 parts to: /content/discord_parts
01/17: 1191 chars | ## #biblically-accurate-press\n\n### 1) Key proposals\n- Form an “Alliance of the Wheel” / Wheel peace pact to prevent infighting and expand outward. (electrobis, Sacland)\n- Adopt Wheel “constitution” principles: democratic decision-making, no fratricide, mutual defense/collective retaliation for stabs...
02/17: 1548 chars | ### 2) Confirmed commitments\n- Public support expressed for the proposed Wheel alliance/peace pact. (credencetothekind, sanserino34, ronnec_15723)\n- Direapache initially planned a fleet build, then switched to building an army after realizing the location is landlocked. (direapache)\n- Dorothynquisit...
03/17: 1121 chars | ### 4) Notable claims / intel\n- STFU status update: Bursa, Grouse, Erzurum, and the speaker allied; Kayseri wants to join. (epicsquidmoment + listed)\n- “Seljuk STFU alliance” consolidation claimed: Bursa, Grouse, Erzurum, Diyarbakır; said they can be friendly with others. (epicsqu

In [ ]:
import shutil
zip_path = shutil.make_archive("discord_parts", "zip", root_dir="discord_parts")
zip_path


'/content/discord_parts.zip'

In [ ]:
from google.colab import files
files.download("discord_parts.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

BASE = merged.copy()  # or all_msgs.copy()

BASE["timestamp"] = pd.to_datetime(BASE["timestamp"])
BASE["date"] = BASE["timestamp"].dt.date
BASE["hour"] = BASE["timestamp"].dt.floor("H")

total_msgs = len(all_msgs)
total_turns = len(merged)
unique_speakers = BASE["author_id"].nunique()
channels_rank = BASE.groupby("channel").size().sort_values(ascending=False)

top_speakers = (BASE.groupby("author_name")
                .size()
                .sort_values(ascending=False)
                .head(10))

print("Total messages:", total_msgs)
print("Merged turns:", total_turns)
print("Unique speakers:", unique_speakers)
print("\nMost active channels:")
print(channels_rank.head(10))
print("\nTop speakers (by merged turns):")
print(top_speakers)


Total messages: 22530
Merged turns: 10144
Unique speakers: 134

Most active channels:
channel
riftposting                  3044
joining-the-hyperwar         1989
torus-supremacy              1662
hyperpeace-in-our-time       1420
stab-central                 1084
biblically-accurate-press     904
sea-of-ice                     41
dtype: int64

Top speakers (by merged turns):
author_name
vriska2195             409
jes_real               344
aeolus_mare            316
direapache             307
emperornaxos           300
styxsunset             250
jackofalltrades2751    244
nevjev                 239
me18guy                220
huxrax                 217
dtype: int64


/tmp/ipython-input-1191320098.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  BASE["hour"] = BASE["timestamp"].dt.floor("H")


In [ ]:
busiest_hour = BASE.groupby("hour").size().sort_values(ascending=False).head(5)
print("Busiest hours (overall):")
print(busiest_hour)

busiest_by_channel = (BASE.groupby(["channel","hour"]).size()
                      .reset_index(name="turns")
                      .sort_values(["channel","turns"], ascending=[True, False]))
print("\nBusiest hour per channel:")
display(busiest_by_channel.groupby("channel").head(1))


Busiest hours (overall):
hour
2026-01-17 21:00:00+01:00    4437
2026-01-17 22:00:00+01:00    3258
2026-01-17 23:00:00+01:00    2449
dtype: int64

Busiest hour per channel:


,channel,hour,turns
0,biblically-accurate-press,2026-01-17 21:00:00+01:00,357
3,hyperpeace-in-our-time,2026-01-17 21:00:00+01:00,705
7,joining-the-hyperwar,2026-01-17 22:00:00+01:00,763
9,riftposting,2026-01-17 21:00:00+01:00,1360
12,sea-of-ice,2026-01-17 21:00:00+01:00,27
15,stab-central,2026-01-17 21:00:00+01:00,413
18,torus-supremacy,2026-01-17 21:00:00+01:00,835


In [ ]:
event_type_counts = events_df["type"].value_counts()
events_by_channel = (events_df.groupby("channel").size()
                     .sort_values(ascending=False)
                     .head(10))

print("Event types:")
print(event_type_counts)

print("\nEvents per channel (top 10):")
print(events_by_channel)


Event types:
type
info          404
proposal      277
request       204
commitment    115
meta           69
warning        65
Name: count, dtype: int64

Events per channel (top 10):
channel
riftposting                  320
joining-the-hyperwar         218
torus-supremacy              198
hyperpeace-in-our-time       157
stab-central                 123
biblically-accurate-press    111
sea-of-ice                     7
dtype: int64


In [ ]:
replies = all_msgs.dropna(subset=["reply_to"])
reply_counts = replies.groupby(["channel","reply_to"]).size().reset_index(name="reply_count")

key_msgs = (all_msgs.merge(reply_counts, how="left",
                           left_on=["channel","message_id"],
                           right_on=["channel","reply_to"])
            .fillna({"reply_count": 0})
            .sort_values("reply_count", ascending=False))

top_replied = key_msgs.head(10)[["channel","timestamp","author_name","reply_count","content"]]
top_replied


,channel,timestamp,author_name,reply_count,content
2164,hyperpeace-in-our-time,2026-01-17 21:00:17.906000+01:00,andrium,20.0,@Eskişehir @Istanbul @Adana @Partridge @Pheasa...
2165,hyperpeace-in-our-time,2026-01-17 21:00:17.906000+01:00,andrium,20.0,@Eskişehir @Istanbul @Adana @Partridge @Pheasa...
20289,torus-supremacy,2026-01-17 21:39:42.539000+01:00,radiolurken,16.0,france alliance that will collapse in 15 minutes
20288,torus-supremacy,2026-01-17 21:39:42.539000+01:00,radiolurken,16.0,france alliance that will collapse in 15 minutes
7386,joining-the-hyperwar,2026-01-17 22:16:37.959000+01:00,armygeneralf,14.0,updated map
7387,joining-the-hyperwar,2026-01-17 22:16:37.959000+01:00,armygeneralf,14.0,updated map
16137,stab-central,2026-01-17 21:00:17.969000+01:00,.mcmoney,12.0,@Chicken\n@Fireback\n@Peacock\n@Quail\n@Grouse...
16136,stab-central,2026-01-17 21:00:17.969000+01:00,.mcmoney,12.0,@Chicken\n@Fireback\n@Peacock\n@Quail\n@Grouse...
14152,riftposting,2026-01-17 22:50:44.907000+01:00,andrium,12.0,please CAN ANYBODY TELL ME WHAT IS GOING ON IN...
14153,riftposting,2026-01-17 22:50:44.907000+01:00,andrium,12.0,please CAN ANYBODY TELL ME WHAT IS GOING ON IN...


In [ ]:
def stats_block():
    top_channels = channels_rank.head(5)
    top_speak = top_speakers.head(5)
    top_hours = busiest_hour.head(3)

    lines = []
    lines.append("### Digest stats (FYI)")
    lines.append(f"- Total messages: **{len(all_msgs)}** | Merged turns: **{len(merged)}** | Unique speakers: **{unique_speakers}**")
    lines.append("- Most active channels: " + ", ".join([f"**#{ch}** ({n})" for ch, n in top_channels.items()]))

    lines.append("- Top speakers (merged turns): " + ", ".join([f"**{name}** ({n})" for name, n in top_speak.items()]))

    lines.append("- Busiest hours: " + ", ".join([f"**{h}** ({n})" for h, n in top_hours.items()]))

    if "type" in events_df.columns:
        etc = event_type_counts
        lines.append("- Event mix: " + ", ".join([f"**{k}** {int(v)}" for k, v in etc.items()]))

    lines.append("")  # spacer
    return "\n".join(lines)

stats_md = stats_block()
print(stats_md)


### Digest stats (FYI)
- Total messages: **22530** | Merged turns: **10144** | Unique speakers: **134**
- Most active channels: **#riftposting** (3044), **#joining-the-hyperwar** (1989), **#torus-supremacy** (1662), **#hyperpeace-in-our-time** (1420), **#stab-central** (1084)
- Top speakers (merged turns): **vriska2195** (409), **jes_real** (344), **aeolus_mare** (316), **direapache** (307), **emperornaxos** (300)
- Busiest hours: **2026-01-17 21:00:00+01:00** (4437), **2026-01-17 22:00:00+01:00** (3258), **2026-01-17 23:00:00+01:00** (2449)
- Event mix: **info** 404, **proposal** 277, **request** 204, **commitment** 115, **meta** 69, **warning** 65

